In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def patchify(imgs, side_length):
  """
  imgs: B, C, H, W
  sidelengt: length of each paths
  """
  B, C, H, W = imgs.shape
  assert H % side_length == 0
  assert W % side_length == 0

  no_patches_h = H // side_length
  np_patches_w = W // side_length

  patches = []
  for h_idx in range(no_patches_h):
    for w_idx in range(np_patches_w):
      patch = imgs[:, :, h_idx * side_length:(h_idx+1) * side_length, w_idx * side_length:(w_idx+1) * side_length]
      patches.append(patch) # B, C, side_length, side_length

  patches = torch.stack(patches, dim=1) # B, P, C, H, W
  return patches



In [ ]:
class SingleHeadAttention(nn.Module):
  def __init__(self, n_embd, head_size):
    super().__init__()
    self.n_embd = n_embd
    self.head_size = head_size
    self.q = nn.Linear(n_embd, head_size, bias=False)
    self.k = nn.Linear(n_embd, head_size, bias=False)
    self.v = nn.Linear(n_embd, head_size, bias=False)

  def forward(self, x):
    """
    x: B, P, n_embd
    """
    B, P, n_embd = x.shape
    assert n_embd == self.n_embd

    q_val = self.q(x) # B, P, n_embd
    k_val = self.k(x) # B, P, n_embd
    v_val = self.v(x) # B, P, n_embd

    sim = (q_val @ k_val.permute(0, 2, 1)) * self.head_size**(-0.5) # B, P, P
    wei = F.softmax(sim, dim=-1)
    res = wei @ v_val # B, P, n_embd
    return res







In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, n_embd, no_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([SingleHeadAttention(n_embd, head_size) for _ in range(no_heads)])
    self.proj = nn.Linear(no_heads * head_size, n_embd)

  def forward(self, x):
    out = torch.concat([head(x) for head in self.heads], dim=-1)
    out = self.proj(out)

    return out

In [ ]:
class AttentionBlock(nn.Module):
  def __init__(self, n_embd, no_heads):
    super().__init__()
    assert n_embd % no_heads == 0
    head_size = n_embd // no_heads
    mlp_ratio = 4

    self.msa = MultiHeadAttention(n_embd, no_heads, head_size)
    self.ff = nn.Sequential(
        nn.Linear(n_embd,mlp_ratio* n_embd),
        nn.GELU(),
        nn.Linear(mlp_ratio*n_embd, n_embd)
    )

    self.ln_1 = nn.LayerNorm(n_embd)
    self.ln_2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    """
    x: B, P, n_embd
    """
    z_dash = self.msa(self.ln_1(x)) + x
    z = self.ff(self.ln_2(z_dash)) + z_dash

    return z


In [ ]:
class ViT(nn.Module):
  def __init__(self, patch_size, no_patches, n_embd, no_blocks, no_heads, no_classes):
    super().__init__()
    self.pos_embd_tabl = nn.Embedding(no_patches, n_embd)
    self.patch_proj = nn.Linear(patch_size, n_embd, bias=False)
    self.zero_embd = nn.Parameter(torch.rand(n_embd))

    self.blocks = nn.Sequential(*[AttentionBlock(n_embd, no_heads) for _ in range(no_blocks)])
    self.classifier = nn.Linear(n_embd, no_classes)

  def forward(self, x):
    """
    x: B, P, patch_size: Beware flattening the imgs must be done before
    """
    B, P, patch_size = x.shape

    pos_embd = self.pos_embd_tabl(torch.arange(P, device=x.device)) # P
    input_proj = self.patch_proj(x) # B, P, n_embd
    x = input_proj + pos_embd
    zero_embd_tiled = self.zero_embd[None, None, :].expand(B, 1, -1)
    x = torch.concat([zero_embd_tiled, x], dim=1) # B, (P+1), n_embd
    x = self.blocks(x)# B, (P+1), n_embd
    x_0 = x[:, 0, :]

    x = self.classifier(x_0)

    return x

In [ ]:
import numpy as np
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.datasets.mnist import MNIST
from torchvision.transforms import ToTensor
from tqdm import tqdm, trange

np.random.seed(0)
torch.manual_seed(0)

In [ ]:
train_set = MNIST("./datasets", train=True, download=True, transform=ToTensor())
test_set = MNIST("./datasets", train=False, download=True, transform=ToTensor())

B = 128
train_loader = DataLoader(train_set, shuffle=True, batch_size=B)
test_loader = DataLoader(test_set, shuffle=False, batch_size=B)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

patch_side_length = 4
no_patches = 49

model = ViT(patch_size=patch_side_length**2, no_patches=no_patches, no_blocks=2, n_embd=8, no_heads=2, no_classes=10).to(device)

N_EPOCHS = 5
LR = 0.005

optimizer = Adam(model.parameters(), lr=LR)
criterion = CrossEntropyLoss()

for epoch in trange(N_EPOCHS, desc="Training"):
  train_loss = 0.0
  for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1} in training", leave=False):
    imgs, labels = batch
    imgs, labels = imgs.to(device), labels.to(device)
    imgs_patchified = patchify(imgs, patch_side_length)
    imgs_patchified = imgs_patchified.flatten(2, -1)

    preds = model(imgs_patchified)
    loss = F.cross_entropy(preds, labels)

    train_loss += loss.detach().cpu().item() / len(train_loader)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(f"Epoch {epoch + 1}/{N_EPOCHS} loss: {train_loss:.2f}")


# Test loop
with torch.no_grad():
  correct, total = 0,0
  test_loss = 0.0
  for batch in tqdm(test_loader, desc="Testing"):
    x,y = batch
    x, y = x.to(device), y.to(device)
    y_hat = model(x)
    loss = F.cross_entropy(y_hat, y)
    test_loss += loss.detach().cpu().item() / len(test_loader)
    correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()

    total += len(x)
  print(f"Test loss: {test_loss:.2f}")
  print(f"Test accuracy: {correct / total * 100:.2f}%")





Using device cuda


Training:  20%|██        | 1/5 [00:11<00:45, 11.28s/it]

Epoch 1/5 loss: 1.20



Training:  40%|████      | 2/5 [00:21<00:32, 10.91s/it]

Epoch 2/5 loss: 0.51


...

Epoch 3/5 loss: 0.38



Training:  80%|████████  | 4/5 [00:43<00:10, 10.94s/it]

Epoch 4/5 loss: 0.33



Training: 100%|██████████| 5/5 [00:54<00:00, 10.99s/it]

Epoch 5/5 loss: 0.31


Testing: 100%|██████████| 79/79 [00:01<00:00, 63.14it/s]

Test loss: 0.25
Test accuracy: 92.01%
